In [8]:
from github import Github
from scipy import linalg
# from commonjs_package_tokenizer import tokenize_package
import npm_crawler as npm
import os
import commonjs_package_tokenizer as t
import packages_term_indexer as pti_mod
import pandas as pd


In [59]:
%%time

# spider npm's "most starred repositories" page for
# package name and a github url.
packages = npm.get_most_starred_packages(max_pages=10)

# providing a github login means we can exceed the public API request limit of 60 requests per hour.
# We need to look up approximately 400 packages.
gh_user = os.environ.get('GITHUB_API_USER')
gh_token = os.environ.get('GITHUB_API_TOKEN')
gh = Github(gh_user, gh_token)

# for each of those packages, perform a shallow git clone
for name, gh_url in packages:
    clone_url = 'git://github.com/{}.git'.format(gh_url)
    clone_dir = '.cache/packages/{}'.format(name)
    
    # use github API to determine if this repo is predominantly javascript.
    # we want to exclude coffeescript/typescript repos because their tokens
    # are significantly different.
    is_javascript = gh.is_repo_javascript(gh_url)
    
    if is_javascript and not isdir(clone_dir):
        # git clone the repo. assign output to an unused variable to supress noise
        _ = !git clone $clone_url $clone_dir --depth=1

CPU times: user 652 ms, sys: 55.7 ms, total: 707 ms
Wall time: 1.06 s


In [72]:
%%time
reload(t)
reload(pti_mod)

# build a document-term matrix for all the packages we downloaded
pti = pti_mod.PackagesTermIndexer()

i = 0
for dir_name in os.listdir('.cache/packages'):
    if dir_name == '.gitkeep': continue
    i += 1
    if i > 30: break
    pkg = pti.register_package(dir_name)
    for token in t.tokenize_package(dir_name):
        pkg.register_term(token)

CPU times: user 12.4 s, sys: 384 ms, total: 12.8 s
Wall time: 15.1 s


In [73]:
%%time

wf_matrix = pti.word_frequency_matrix()
tfidf_matrix = pti.tfidf_matrix()

CPU times: user 3.19 s, sys: 100 ms, total: 3.29 s
Wall time: 3.33 s


In [74]:
df = pd.DataFrame(wf_matrix, columns=pti.term_indices, index=pti.package_names)
df.head()

,var,agenda,=,module.exports,require(./lib/agenda.js);,/*,Code,forked,from,https://github.com/rschmukler/agenda,...,value.substring(lastStart);,lastCharacter,lastValue[lastValue.length,(!includeSeparator,separator.test(lastCharacter),"lastValue.substring(0,",lastValue.length,tokens.push(lastValue);,tokens;,require(./cli);
agenda,22,2,95,3,1,2,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0
angular,4458,0,6130,9,0,42,0,0,64,0,...,0,0,0,0,0,0,0,0,0,0
angular2,323,0,482,28,0,2,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
apn,75,0,181,14,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
async,92,0,159,3,0,1,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
df = pd.DataFrame(tfidf_matrix, columns=pti.term_indices, index=pti.package_names)
df.head()

,var,agenda,=,module.exports,require(./lib/agenda.js);,/*,Code,forked,from,https://github.com/rschmukler/agenda,...,value.substring(lastStart);,lastCharacter,lastValue[lastValue.length,(!includeSeparator,separator.test(lastCharacter),"lastValue.substring(0,",lastValue.length,tokens.push(lastValue);,tokens;,require(./cli);
agenda,4.367034,1.584963,6.584963,2.000000,1,1.584963,1,1,1.584963,1,...,0,0,0,0,0,0,0,0,0,0
angular,11.703032,0.000000,12.581907,3.321928,0,5.426265,0,0,6.022368,0,...,0,0,0,0,0,0,0,0,0,0
angular2,8.051268,0.000000,8.915879,4.857981,0,1.584963,0,0,2.584963,0,...,0,0,0,0,0,0,0,0,0,0
apn,6.031732,0.000000,7.507795,3.906891,0,0.000000,0,0,2.584963,0,...,0,0,0,0,0,0,0,0,0,0
async,6.312886,0.000000,7.321928,2.000000,0,1.000000,0,0,2.807355,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
%%time
T,sigma,D_trans = linalg.svd(tdm, full_matrices=False)

CPU times: user 434 ms, sys: 24.2 ms, total: 458 ms
Wall time: 405 ms


In [51]:
pd.DataFrame(D_trans, columns=pti.term_indices, index=pti.package_names)

ValueError: Shape of passed values is (84255, 30), indices imply (84256, 30)